In [38]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier, Pool

PATH = 'smm_baseline/sample_data/fold1/fit/'
PATH_PRED = 'smm_baseline/sample_data/fold1/predict/test_users.parquet'

In [39]:
# Загрузка данных
smm_train_events = pd.read_parquet(PATH + "smm_train_events.parquet")
user_features    = pd.read_parquet(PATH + "user_features.parquet")
user_tags        = pd.read_parquet(PATH + "user_tags.parquet")
test_users       = pd.read_parquet(PATH_PRED)

In [40]:
# Определение временных границ для тренировочного и тестового наборов данных
train_end_date = smm_train_events['datetime'].min() + pd.DateOffset(weeks=4)
train_data = smm_train_events[smm_train_events['datetime'] <= train_end_date]
test_data = smm_train_events[(smm_train_events['datetime'] > train_end_date) & (smm_train_events['datetime'] <= train_end_date + pd.DateOffset(weeks=4))]

# Фильтрация пользовательских характеристик по id из smm_train_events
user_ids_unique = smm_train_events['user_id'].unique()
user_tags = user_tags[user_tags['user_id'].isin(user_ids_unique)]

# Фильтр по юзерам + заполнение пропусков средним
user_features = user_features[user_features['user_id'].isin(user_ids_unique)].fillna(user_features.mean())

# Извлечение признаков на основе истории покупок
event_counts = train_data.groupby(['user_id', 'item_id'])['event'].count().reset_index().rename(columns={'event': 'event_count'})
unique_items = train_data.groupby('user_id')['item_id'].nunique().reset_index().rename(columns={'item_id': 'unique_items_count'})
unique_categories = train_data.groupby('user_id')['category_id'].nunique().reset_index().rename(columns={'category_id': 'unique_categories_count'})
#features_from_events = pd.merge(event_counts, unique_items, on='user_id')
# features_from_events = pd.merge(features_from_events, unique_categories, on='user_id')
features_from_events = pd.merge(unique_items, unique_categories, on='user_id')

# Извлечение признаков из user_tags
unique_tags = user_tags.groupby('user_id')['user_tag'].nunique().reset_index().rename(columns={'user_tag': 'unique_tags_count'})
avg_tag_level = user_tags.groupby('user_id')['tag_level'].mean().reset_index().rename(columns={'tag_level': 'avg_tag_level'})
avg_tag_score = user_tags.groupby('user_id')['tag_score'].mean().reset_index().rename(columns={'tag_score': 'avg_tag_score'})

# Объединение признаков из user_tags в один датафрейм
features_from_tags = pd.merge(unique_tags, avg_tag_level, on='user_id')
features_from_tags = pd.merge(features_from_tags, avg_tag_score, on='user_id')

# Объединение всех признаков в один датафрейм
combined_features = pd.merge(user_features, features_from_events, on='user_id', how='left')
combined_features = pd.merge(combined_features, features_from_tags, on='user_id', how='left')

# Заполнение пропущенных значений после объединения
combined_features = combined_features.fillna(0)

# Формирование датасета для обучения
train_data = train_data.merge(event_counts, on=['user_id', 'item_id'], how='left')
train_dataset = pd.merge(train_data, combined_features, on='user_id', how='left')
train_dataset['target'] = ((train_dataset['event'] == 2) | (train_dataset['event_count'] > 1)).astype(int)

# Замена NaN значений на 0
train_dataset.fillna(0, inplace=True)

# Выбор признаков и целевой переменной
features = train_dataset.columns.difference(['user_id', 'item_id', 'category_id', 'event', 'datetime', 'report_dt', 'f163', 'target'])
X = train_dataset[features]
y = train_dataset['target']

# Чистим память
del train_end_date, user_ids_unique, user_tags, user_features, event_counts, unique_items, unique_categories, features_from_events, unique_tags, avg_tag_level, avg_tag_score, features_from_tags, combined_features

/var/folders/x0/bzrtlp_150sb29kprmm15lkr0000gn/T/ipykernel_14579/2215692897.py:11: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  user_features = user_features[user_features['user_id'].isin(user_ids_unique)].fillna(user_features.mean())
/var/folders/x0/bzrtlp_150sb29kprmm15lkr0000gn/T/ipykernel_14579/2215692897.py:11: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  user_features = user_features[user_features['user_id'].isin(user_ids_unique)].fillna(user_features.mean())


In [42]:
# Обучение модели
cat = True
if cat:
    model = CatBoostClassifier(iterations=5, learning_rate=0.01, random_seed=42)
    model.fit(Pool(X[features], y))

    #Проверка важности признаков
    feature_importance = pd.DataFrame({
        'feature': features,
        'importance': model.get_feature_importance()
    }).sort_values(by='importance', ascending=False)

    features = feature_importance.iloc[:10, 0].tolist()
    model.fit(Pool(X[features], y))

else:

    model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    model.fit(X, y)

    # Оценка важности признаков
    feature_importance_rf = pd.DataFrame({
        'feature': features,
        'importance': model.feature_importances_
    }).sort_values(by='importance', ascending=False)

    features = feature_importance.iloc[:10, 0].tolist()
    model.fit(Pool(X[features], y))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


0:	learn: 0.6578093	total: 87.9ms	remaining: 352ms
1:	learn: 0.6243992	total: 146ms	remaining: 219ms
2:	learn: 0.5914506	total: 199ms	remaining: 133ms
3:	learn: 0.5603694	total: 247ms	remaining: 61.6ms
4:	learn: 0.5312665	total: 298ms	remaining: 0us
0:	learn: 0.6842956	total: 12.2ms	remaining: 48.6ms
1:	learn: 0.6757243	total: 20.4ms	remaining: 30.6ms
2:	learn: 0.6672903	total: 26.3ms	remaining: 17.5ms
3:	learn: 0.6589764	total: 34.5ms	remaining: 8.62ms
4:	learn: 0.6508508	total: 42.8ms	remaining: 0us


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [43]:
# Формирование датасета для предсказаний
top_predictions = pd.DataFrame()

for user in test_users['user_id'].unique():
    prediction_df = []
    user_features = train_dataset[train_dataset['user_id'] == user][features + ['user_id', 'item_id']]
    for item in train_data['item_id'].unique():
        prediction_df.append({'user_id': user, 'item_id': item, 'probability': 0})
    
    prediction_df = pd.DataFrame(prediction_df)
    prediction_df = pd.merge(prediction_df, user_features, on=['user_id','item_id'], how='left').fillna(0)
    
    # Предсказание вероятностей покупки
    prediction_df['probability'] = model.predict_proba(prediction_df[features])[:, 1]
    prediction_df = prediction_df[['user_id', 'item_id', 'probability']]
    
    # Выбор 20 товаров с наивысшей вероятностью покупки для каждого пользователя
    prediction_df = prediction_df.groupby('user_id').apply(lambda x: x.nlargest(20, 'probability')).reset_index(drop=True)
    top_predictions = pd.concat([top_predictions, prediction_df[['user_id', 'item_id']]], ignore_index=True)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.fram

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.fram

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.fram

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.fram

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.fram

...

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.fram

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.fram

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.fram

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Library/Frameworks/Python.fram

,user_id,item_id
0,6887097,212755
1,6887097,212755
2,6887097,1815586
3,6887097,1815586
4,6887097,1530517
5,6887097,1530517
6,6887097,1230582
7,6887097,1230582
8,6887097,561794
9,6887097,561794


In [ ]:
# Сохранение результатов в файл
top_predictions.to_csv('result.csv', index=False)